## Imports

In [ ]:
!pip3 install snntorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
from snntorch import spikegen
import torch

## Leaky Integrate-and-Fire (LIF) Neuron Model


- Suma (weight, input) apare si in cazul neuronilor spike
- Nu se foloseste functie de activare
- Suma (weight, input) contribuie la potentialul de membrana - U(t)
- Daca suma (weight, input) depaseste un threshold, neuronul emite un spike
- Input-ul unui neuron reprezinta impulsuri scurte de electricitate
- Apare o problema deoarece este improbabil ca neuronul sa primeasca la un moment dat mai multe spike-uri input
- Asadar avem nevoie de o persistenta temporala. Vrem sa retinem potentialul membranei in timp
---
- **Formula potential membrana:** U[t] = βU[t − 1] + WX[t] − S_out[t − 1]θ *(Eq4)*
  - U[t] = potentialul membranei la momentul de timp t
  - β = e^−1/τ = decay rate / rata de decadere
  - WX[t] = inmultirea dintre wights si input-urile neuronului
  - θ = threshold-ul neuronului
  - S_out [t] ∈ {0, 1} = output-ul neuronului
  - S_out [t] = 1 if U [t] > θ, 0 otherwise
  - S_out[t − 1]θ = daca neuronul a emis un spike, potentialul membranei trebuie resetat *(Eq5)*
---
- **Resetarea soft** = βS_out[t − 1]θ
  - Daca se depaseste treshold-ul, nu resetam potentialul membranei la zero
  - Ofera performante mai bune(exemplu: acuratete)
  - Inca nu se cunoaste de ce ofera performante mai bune
- **Resetarea hard(reset-to-zero)** = S_out[t − 1]θ
  - Daca se depaseste threshold-ul, resetam potentialul membranei la zero
---
**Exemplu implementare neuron LIF**

In [ ]:
def lif(X, U):
  beta = 0.9 # set decay rate
  W = 0.5 # learnable parameter
  theta = 1 # set threshold
  S = 0 # initialize output spike

  U = beta * U + W * X - S * theta # iterate over one time step of Eq. 4
  S = int(U > theta) # Eq. 5
  return S, U

**Exemplu neuron LIF snnTorch**

In [ ]:
import snntorch as snn

lif = snn.Leaky(beta=0.9, threshold=1) # initialize neuron

nr_iterations = 10
for i in range(nr_iterations):
  S, U = lif(X * W, U) # Eq.4 and Eq. 5 are recurrently returned
  print(f"Iter {i}: S = {S}, U = {U}")

**5 input-uri cu 5 neuroni LIF - fiecare input este legat de un singur neuron LIF**

In [ ]:
import torch

lif = snn.Leaky(beta=1.0, threshold=1) # initialize neuron

X = torch.rand(5) # vector of 10 random inputs
U = torch.zeros(5) # initialize hidden states of 10 neurons to 0 V

print("X=", X, end="\n\n")

nr_iterations = 0
for i in range(nr_iterations):
  S, U = lif(X, U) # forward-pass of leaky integrate-and-fire neuron
  print("Iter:", i)
  print("S:", S)
  print("U:", U)
  print()

X= tensor([0.8809, 0.1665, 0.1596, 0.0920, 0.5783])



## Input encoding


- Convertirea datelor in spike-uri ce sunt trimise ca input retelei neuronale
- Input-ul retelei neuronale nu sunt neaparat spike-uri, pot fi si valori continue
- Exista 3 codari populare
- 1) **Rate encoding**
  - Input mare - multe spike-uri
  - Input mic - putine spike-uri
  - Intr-un inverval de x steps, input-ul mare va produce mai multe spike-uri decat un input mic
- 2) **Latency Coded Inputs**
  - Input mare - spike rapid
  - Input mic - spike intarziat
  - Numarul spike-urilor nu mai conteaza
  - Conteaza cand apare spike-ul
  - In comparatie cu Rate Encoding, aceasta metoda atribuie mai multa
  importanta fiecarui spike
  - Intr-un inteval de x steps, input-ul mare va produce spike in primul step, iar input-ul mic la urma
- 3) **Delta Modulated Inputs**
  - Spike-uri doar cand se produce o schimbare
  - Daca nu apare vreo schimbare, probabilitatea ca un spike sa apara este mica
  - Spike doar daca diferenta dintre doua input-uri din perioade consecutive de timp este mai mare decat un threshold
- Exemplele de mai sus convertesc date normale la spike-uri
- Este mai eficient sa obtinem spike-uri natural, fara conversie (exemplu: camera DVS capteaza schimbarile din mediu folosind delta modulation)
- In procesul de convertire al input-ului la spike-uri, se pierde informatie
- Daca converitrea nu se poate evita, se recomanda rate encoding
- Ideal este ca senzorii sa captureze informatia sub forma de spike-uri, pentru a nu fi necesara conversia si compresia datelor  

**Rate encoding**

In [ ]:
steps = 20 # number of time steps

X = torch.rand(10) # vector of 10 random inputs
S = spikegen.rate(X, num_steps=steps)

print(X.size())
print(S.size())
# print(X)
# print(S)

torch.Size([10])
torch.Size([20, 10])


**Latency Coded Inputs**

In [ ]:
steps = 13 # number of time steps

X = torch.rand(10) # vector of 10 random inputs
X[0] = 0.01
X[1] = 0.02
S = spikegen.latency(X, num_steps=steps)
print(X.size())
print(S.size())
# print(X)
# print(S)

torch.Size([10])
torch.Size([13, 10])


**Delta Modulated Inputs**

In [ ]:
print(X.size())

S = spikegen.delta(X, threshold=0.8) # convert X to delta modulated spikes in S

print(S.size())# no change to the size; only to the elements
print(X)
print(S)

torch.Size([10])
torch.Size([10])
tensor([0.0100, 0.0200, 0.8423, 0.8253, 0.4189, 0.1415, 0.1781, 0.8496, 0.8847,
        0.0609])
tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])


## Output decoding

- Convertirea output-ului in informatie relevanta
- In contextul antrenarii retelelor neuronale spike, input endocoding nu constrange ce output decoding putem folosi
- Exista 3 decodari:
- 1) **Rate coding**
  - Alegem neuronul din stratul output care a generat cele mai multe spike-uri
  - Consideram problema clasificarii cu N clase.
  - O retea neuronala non-spike alege clasa asociata neuronului cu cel mai mare output in urma functiei de activare
  - O retea neuronala spike alege clasa asociata neoronului cu cea mai mare frecventa de spiking
  - Neuronul este simulat de X ori, se alege neuronul care a produs de cele mai multe spike-uri
  - Avantaje:
    - Toleranta de eroare: se produc multe spike-uri, asadar nu este o problema daca la un moment dat neuornul nu reuseste sa produca unul
    - Mai multe spike-uri reprezinta mai multa invatare: absenta spike-urilor poate duce la "dead neuron problem"
- 2) **Latency (or temporal) coding**
  - Alegem neuronul care emis primul un spike
  - Rezolva problema consumului ridicat de energie(nu este nevoie de multe spike-uri cum e in cazul rate coding)
  - Avantaje
    - Consum enegie: mai putine spike-uri inseamna mai putina enegie disipata in hardware. De asemenea reduce numarul de accesari din memorie, din cauza sparsitatii
    - Viteza: timpul de reactie al omului este ~250 ms, iar rata de spiking din creier este de oridinul 10Hz, asadar o persoana poate procesa doar 2-3 spike-uri in timpul de reactie. Aceasta problema la rate coding(neuronul trebuie sa produca spike-uri intr-o perioada restrictionata de timp) poate fi adresata introducand mai multi neuroni. Aceasta solutie produce un consum ridicat de enegie. Latency codes foloseste un singur spike pentru reprezentarea informatiei.
  - Rate-coding poate explica doar 15% din activitatea neuronilor in cortexul vizual primar(V1)
  - Daca neuronii din creier ar folosi rate-coding, s-ar consuma cu un ordin de magnitudine mai multa enegie decat codarea temporala

## Functii obiectiv

1) **Spike Rate Objective Functions**
- Multe functii de loss pot fi folosite pentru a incuraja layer-ul de output sa produca spike-uri encodate in rate code sau temporal code
- In general, cross entropy loss sau mean square error sunt aplicate pe numarul spike-urilor sau pe potentialul membranei neuronilor din output
- Cross Entropy Loss
  - Spike Count
    - Incurajeaza ca neuronul clasei corecte sa produca mai multe spike-uri
    - Incurajeaza ca neuronii claselor incorecte sa produca mai putine spike-uri
  - Membrane Potential
    - Incurajeaza ca neuronul clasei corecte sa aiba potentialul membranei ridicat, rezultand spike-uri mai regulate
- Mean Square Error
  - Spike Count
    - Mean square error este aplicat pe numarul spike-urilor fiecarui neuron si numarul spike-urilor target al fiecarui neuron
    - In practica, se foloseste o proportie a numarului de spike-uri pe o perioada de timp: clasa corecta ar trebui sa produca spike 80% din timp, in timp ce clasa incorecta 20% din timp
  - Membrane Potential
    - Fiecare neuron output are un potential de membrana target pentru fiecare pas. Eroarea este insumata peste timp si neuroni
- Cu un numar suficient de mare de pasi, Spike Count este mai des folosit pentru functia de loss

2) **Spike Time Objectives**
- Mai putin utilizate



**Loss functions in snnTorch**

In [ ]:
from snntorch import functional as SF

loss_1 = SF.ce_rate_loss() # cross-entropy spike rate
loss_2 = SF.mse_rate_loss() # mean square spike rate
loss_3 = SF.ce_max_membrane_loss() # maximum membrane
loss_4 = SF.mse_membrane_loss() # mean square membrane

**Spike Time Objectives in snnTorch**

In [ ]:
from snntorch import functional as SF

loss_1 = SF.ce_temporal_loss() # cross-entropy spike time
loss_2 = SF.mse_temporal_loss() # mean square spike time
loss_3 = SF.mse_membrane_loss() # mean square membrane - target must be latency-coded

## Learning rules

- Dupa ce s-a ales functia de loss, aceasta trebuie folosita pentru a actualiza parametrii
- **Perturbation learning**
  - Se perturba weight-urile pentru a observa schimbarea erorii
  - Daca eroarea s-a micsorat, perturbarea este acceptata, altfel este respinsa
  - Are rezultate daca avem un numar foarte mare de incercari, ceea ce nu este practic
  - Dificultatea invatarii creste odata cu numarul de weight-uri
- **Random Feedback**
  - In backpropagation eroarea este transportata de la un strat la altul
  - Eroarea este multiplicata cu weight-ul fiecarui strat
  - A aratat performante similare cu backopropagation in retele simple
  - Tot feedback-ul din backpropagare este inlocuit cu unul random
- **Local Losses**
  - Fiecare strat are propria functie de loss
  - Aceasta metoda se bazeaza pe faptul ca retelel neuronale shallow sunt mai usor de antrenat
- **Forward-Forward Error Propagation**
  - Pasul de backpropagare este inlocuit cu un alt forward-pass, unde semnalul de input este alterat in functie de eroare

## Activity Regularisation

- Functiile de loss cu rate codes cresc potentialul membranei si frecventa spike-urilor, fara un prag superior
- Regularizarea loss-ului poate fi folosita pentru a penaliza frecventa crescuta/scazuta a spike-urilor
- Regularizarea scade varianta dar poate creste bias-ul
- Regularizare prea mare poate duce la bias mare
- Regularizarea poate fi aplicata pe neuroni individuali sau pe grupuri de neuroni
- 1) **Population level regularisation**
  - Daca se doreste o eficienta enegetica crescuta, ne intereseaza numarul total de spike-uri din toata reteaua
  - Daca numarul total de spike-uri al ale neuronilor dintr-un start depasesc o anumita valoare, atunci se poate aplica regularizarea
- 2) **Neuron level regularisation**
  - Putem pune pe neuroni limita minima de spike-uri, iar daca neuronul nu produce suficiente spike-uri, sa se aplice regularizarea
- Unul dintre cele mai populare **motive pentru care reteaua neuronala spike nu invata** este numarul scazut de spike-uri al neuronilor
- Sunt necesare teste pentru a observa ce strat are un numar scazut de spike-uri, pentru a se aplica o regularizare
- O solutie simpla ar fi sa scadeam threshold-ul neuronilor din stratul de risc, astfel incat acestia vor produce mai usor spike-uri

## Training Spiking Neural Networks

- Trei metode principale de a antrena retelele neuronale spike
- Rolul metodei backpropagation este de a minimiaza loss-ul
- Pentru a minimiza loss-ul, backpropagation calculeaza derivata loss-ului cu respect fata de fiecare parametru, aplicand regula lantului de la ultimul strat spre primul strat
- Daca gradientul este 0, nu se actualizeaza parametrii
- Problema metodei backpropagation aplicata retelelor neuronale spike consta in non-diferentiabilitatea spike-urilor
- Reamintim formula de calcul al potentialului membranei U [t] = βU [t − 1] + W X[t]
  - Presupunem ca se modifica W => se modica U
  - Daca modificarea lui U nu produce o schimbare de spike, atunci dS/dU = 0
- 1) **Shadow training**
  - O retea neuronala artificiala este antrenata, apoi convertita la una spike
  - Se elimina "dead neuron problem"
  - Sanse mici ca reteaua spike sa ajunga la performanata retelei artificiale intiale
  - Conversia este folositoare atunci cand eficienta in inferenta este mai importanta decat eficienta in antrenare
  - Functia de activare este inlocuita de spike rate sau latency coding
- 2) **Backpropagation Using Spike Times**
  - Reteaua neuronala spike este antrenata folosing backpropagation
  - O metoda de a elimina dead neuron problem este de a calcula derivata in functie de timpul spike-urilor
  - Spike-urile sunt discotinue, timpul este continuu
  - Asadar putem calcula derivata timpului spike-urilor cu respect fata de weight-uri
- 3) **Backpropagation Using Spikes**
  - In pasul forward se foloseste Heavside pe U, pentru a determina daca acel neuron produce spike
  - In pasul de backward se inlocuieste Heavside cu Sigmoid
  - Functia Sigmoid este continua, asadar o putem deriva

- 3.1) **Surrogate Gradients**
- Ne ajuta sa scapam de dead neuron problem
- Formulare dead neuron problem:
  - Potentialul membranei este sub threshold: U < 0
    - Nu se produce spike, derivata este ∂S/∂U = 0
  - Potentialul membranei este peste threshold: U > 0
    - Se produce spike, dar derivata ramane ∂S/∂U = 0
  - Potentialul memebranei este egal cu threshold-ul: U = 0
    - Se produce spike, derivata este ∂S/∂U = oo
- Ajuta ca eroarea sa se propage catre straturi initiale
- Spiking-ul este necesar pentru actualizarea weight-urilor
- Surrogate gradient bun este arctan
- Este surrogat gradient default in ssnTorch
- Nu se stie de ce functioneaza atat de bine
- Surrogate gradients nu pot invata daca nu exista spike-uri
- Asta sublinieaza o importanta distinctie intre dead neuron problem si vanishing gradient problem
  - Un dead neuron este un neuron care nu produce spike-uri => nu contribuie la loss => weight-ul asociat nu are "credit"
  - Vanishing gradients pot aparea si in ANN si in SNN. Apare deoarece weight-urile ajung foarte mici in urma inmultirii repetate cu valori sub 1
- Surrogate gradients nu trebuie specificati explicit in snnTorch, arctan se foloseste by default


**Leaky integrate-and-fire neuron and surrogate gradients in snnTorch**

In [ ]:
import snntorch as snn
from snntorch import surrogate

lif_1 = snn.Leaky(beta=0.9, spike_grad=surrogate.fast_sigmoid())
lif_2 = snn.Leaky(beta=0.9, spike_grad=surrogate.sigmoid())
lif_3 = snn.Leaky(beta=0.9, spike_grad=surrogate.straight_through_estimator())
lif_4 = snn.Leaky(beta=0.9, spike_grad=surrogate.triangular())

## The Link Between Surrogate Gradients and Quantized Neural Networks

- Exista cateva metode de construire a Quantized Neural Networks

1) **Post training quantization**
  - Initial, reteaua este antrenata cu aritmetica pe floating-point
  - Dupa antrenare, weight-urile sunt convertite la precizie mai scazuta
  - Este usor de implementat si eficient
  - Acuratetea poate scadea mult la anumite modele si task-uri

2) **Quantization-aware training**
  - Antrenarea se efectueaza cu cunatizare
  - Procesul de cuantizare este non-diferentiabil, asadar este ignorat in calcularea gradientului aplicand estimatorul Hinton
  - Este mai costisitor computational si poate necesita modificari al algoritmului de antrenare

3) **Mixed-precision training**
  - Diferite parti ale retelei neuronale foloseste diferite nivele de precizie
  - Spre exemplu, in pasul de forward se poate folosi precizie scazuta, iar in pasul de backward, cand se actualizeaza wight-urile se poate folosi precizie ridicata
  - Complexitate computationala scazuta si impact minim asupra acuratetei

4) **Binary and ternay neural networks**
  - Caz extrem de retea cuantizata
  - In reteaua binara weight-urile se convertesc la -1 si 1
  - In reteaua ternata weight-urile se convertesc la -1, 0 si 1
  - Complexitate computationala foarte scazuta
  - Acuratete mica sau model complex


---

Multe imbunatatiri in deep learning sunt rezultatul a multor tehnici de optimizare

Unele dintre aceste tehnici pot fi utilizate in retelele neuronale spike, iar unele sunt specifice doar retelelor neuronale spike

Exemple de tehnici utilizate in retelele neuronale spike


1) **The reset mechanism**
  - Reprezinta termenul din functia spike-ului
  - Nu este diferentiabil
  - Este important sa nu se folosesca in calcularea gradientului deoarece degradeaza performanta
  - Se ignora acel termen in pasul de backward
  - snnTorch face asta automat prin apelarea functiei .detach()

2) **Residual connections**
  - Functioneaza foarte bine pentru retelele neuronale non-spike si spike
  - Se adauga conexiuni intre straturi, sarind peste cateva straturi intermediare
  - Sunt utilizate pentru a rezolva vanishing gradient problem si a imbunatati transmiterea informatiei in pasul de forward si backward
  - Functioneaza bine si in retelele nuronale spike

3) **Learnable decay**
  - In loc sa tratam decay-ul neuronilor ca pe un hiperparametru, il putem face parametru invatabil
  - Seamana mai mult cu retelele neuronale recurente
  - Imbunatateste performanta pe seturi de date variabile in timp

4) **Graded spikes**
  - Fiecare neuron are un parametru invatabil suplimentar
  - Activarea neuronului nu mai este constransa la 0 si 1
  - Complexitatea nu creste foarte mult deoarece numarul de parametrii invatabili creste liniar cu numarul de neuroni

5) **Learnable thresholds**
  - Nu imbunatateste procesul de invatare

6) **Pooling**
  - In retelele neuronale binarizate se foloseste pooling inainte ca valorile activarilor sa treaca prin threshold ca sa devina valori binarizate
  - Corespunde cu a aplica pooling pe potentialul mebranei, insa asta nu imbunatateste performanta retelei
  - Aplicam pooling pe spke-uri
  - Cand mai multe spike-uri apar intr-o fereastra de pooling, majoritatea castiga
  - Se ating performante ridicate

7) **Optimizer**
  - Adam si SGD performeaza bine